In [1]:
import gensim
import numpy as np
import pandas as pd
import pysolr
from gensim.models import Doc2Vec
from transformers import pipeline, set_seed
set_seed(42)
np.random.seed(42)
import sys

In [2]:
# In case your sys.path does not contain the base repo, cd there.
print(sys.path)
%cd 'PATH_OF_BASE_REPO'  # In the solution it will be the path to my repo. This is such that python loads al the files from the top.

['/Users/axelsirota/repos/ml-solr-course/3-query-generation/lab8/solutions', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python37.zip', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7', '/Users/axelsirota/.pyenv/versions/3.7.3/lib/python3.7/lib-dynload', '', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages', '/Users/axelsirota/repos/ml-solr-course/.venv/lib/python3.7/site-packages/IPython/extensions', '/Users/axelsirota/.ipython']
/Users/axelsirota/repos/ml-solr-course


In [3]:
# Use the pipeline method to invoke the text-generation solution with gpt-2 as model.
generator = None
query = 'Midtown sunny two bedroom'
expanded_queries = None #  Generate 10 possible expansions of 50 characters

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [4]:
expanded_queries

[{'generated_text': 'Midtown sunny two bedroom apartments, with two double doors and plenty of private balconies to see your family down in, with the view of the city skyline in just a single window.\n\nGet a look up at the best in trendy boutiques all over the city from the best in Downtown Eastside to the finest in Midtown, with views all over town, and'},
 {'generated_text': "Midtown sunny two bedroom home in the Westside's former downtown development. As you walk down the block, you'll see what looked like a small garage. In its name, it looks like the neighborhood is an abandoned abandoned trailer park. It's a lot smaller than I knew it. Some parts are still abandoned, while others are simply abandoned apartments, like the one you"},
 {'generated_text': 'Midtown sunny two bedroom home in Burbank - with a great view of the bay in the background with a view overlooking the river.\n\n\nLocated on the west side of Santa Monica Blvd and has over 1,300 square feet of living room, living

In [5]:
model_path = '2-ranking/lab4/airbnb_model'
doc2vec_model = Doc2Vec.load(model_path)
print(f'Doc2Vec Model loaded')

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Doc2Vec Model loaded


In [6]:
queries = []
# For each possible expansion, get the similarity with the original query and get the best
for expanded_query in expanded_queries:
    pass
expanded_query = ''

,query,similarity
2,Midtown sunny two bedroom home in Burbank - wi...,0.676217
7,Midtown sunny two bedroom neighborhood that se...,0.573251
0,"Midtown sunny two bedroom apartments, with two...",0.550182
8,Midtown sunny two bedroom condo with four bedr...,0.545813
9,Midtown sunny two bedroom. A single sheet. A b...,0.403343


In [8]:
# Create a client instance.
solr = pysolr.Solr('http://localhost:8983/solr/airbnb', always_commit=True, timeout=10)

In [9]:
non_expanded_results = solr.search(query, **{
                'rows': 100,
            })


In [11]:
expanded_results = solr.search(expanded_query, **{
                'rows': 100,
            })

In [12]:
#Tokenize both the query, get the similarities with both the expanded and not expanded results in the pandas dataframe as always
tokenized_query = None

In [13]:
df_non_expanded_results = pd.DataFrame(non_expanded_results)
# Fill
df_non_expanded_results.sort_values(by="Similarity", ascending=False, inplace=True)


In [14]:
print(f'Most similar document before expansion has description: \n\n{df_non_expanded_results["description"].iloc[0]}\n\nWith similarity: {df_non_expanded_results["Similarity"].iloc[0]}')

Most similar document before expansion has description: 

Bright Two bedroom apartment located in Midtown West

With similarity: 0.8133248686790466


In [15]:
df_expanded_results = pd.DataFrame(expanded_results)
# Fill
df_expanded_results.sort_values(by="Similarity", ascending=False, inplace=True)

In [16]:
print(f'Most similar document after expansion has description: \n\n{df_expanded_results["description"].iloc[0]}\n\nWith similarity: {df_expanded_results["Similarity"].iloc[0]}')

Most similar document after expansion has description: 

Entire one bedroom apartment located in the heart of Manhattan. The alcove living room has floor to ceiling windows and the view on the city and Hudson river.  Doorman building with a gym, rooftop and terrace with sun beds.  Walking distance to Times Square and Port Authority Bus Terminal.<br /><br /><b>The space</b><br />One bedroom apartment located in Midtown West.

With similarity: 0.672989010810852


In [17]:
print(f'Number of documents that surpass 0.5 similarity threshold: {len(df_expanded_results[df_expanded_results["Similarity"] >= 0.5])}')


Number of documents that surpass 0.5 similarity threshold: 16


So it was useless? A key part of our models is that they are trained with our data. I leave you as exercise to see how much it improves if you finetune gpt-2 with a simple dense layer and train that over our airbnb dataset.

Other problem to fix is that our airbnb similarity model is based on our airbnb data, but gpt-2 is generic. We should evaluate word similarity in the same terms, with the glove vectors alone instead. I leave it to you as exercise to see how much that changes as well.


